Testing Pretrained ChatGPT Models for dod parsing
1. text-davinci-003
2. text-curie-001

In [ ]:
import openai
from multiprocessing.dummy import Pool

openai.api_key = "<your api key>"

In [3]:
#testing davinci

title_prompts = ["Parse attributes from: Yashika Shrikhande - Data Analyst II - ZoomInfo - LinkedIn",
                 "Parse attributes from: Henry Schuck - CEO & Founder - ZoomInfo - LinkedIn",
                 "Parse attributes from: Neel Kiran Roshania - Manager, Data Analysis & Applications",
                 "Parse attributes from: Jayson Bailey - Ghana | Professional Profile | LinkedIn.",
                 "Translate and parse attributes from: 贾会民- 深圳赤湾胜宝旺工程有限公司| LinkedIn - 领英"]

location_prompts = ["Parse country, state, city and zip from: McPherson, Kansas, United States ",
                     "Parse country, state, city and zip from:Türkiye ",
                     "Parse country, state, city and zip from: East Falmouth, Massachusetts, United States ",
                     "Parse country, state, city and zip from: Salem, Oregon, United States ",
                     "Parse country, state, city and zip from: Montpellier et périphérie "]

title_translate_classify_prompts = ['find seniority and job department for "sales manager key account zf group"',
                                    'format this job title ":compass: | 𝗗𝗜𝗚𝗜𝗧𝗔𝗟 𝗠𝗔𝗥𝗞𝗘𝗧𝗜𝗡𝗚 𝗖𝗢𝗠𝗣𝗔𝗡𝗬 | :desktop_computer:𝗖𝗢𝗠𝗣𝗟𝗘𝗧𝗘 𝗜.𝗧 𝗦𝗢𝗟𝗨𝗧𝗜𝗢𝗡’𝘀 ｜:bar_chart:𝗛𝗘𝗟𝗣𝗜𝗡𝗚 𝗬𝗢𝗨 𝗧𝗢 𝗚𝗥𝗢𝗪 𝗬𝗢𝗨𝗥 𝗕𝗨𝗦𝗜𝗡𝗘𝗦𝗦| :iphone: :computer: :point_right: 𝗗𝗠 𝗠𝗘 𝗧𝗢 𝗗𝗜𝗦𝗖𝗨𝗦𝗦 𝗡𝗢𝗪."',
                                    'translate and classify this title "compass: Spécialiste en Copywriting & Capitaine du Black Nugget :black_circle: | :boom: On fait décoller vos ventes grâce à des emails et des lancements bien ciblés :rocket:"']

def complete_prompt(prompt):
    completion = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1000,
        temperature=0.5,
        n=1,
        stop=None
    )
    return completion


if __name__ == "__main__":
    with Pool() as pool:
        results = pool.map(complete_prompt, title_prompts)
        for res in results:
            print(res.choices[0].text) #for just the parsed results
#             print(res) #for the entire json object
    




Name: Yashika Shrikhande 
Position: Data Analyst II 
Company: ZoomInfo 
Platform: LinkedIn


Name: Henry Schuck
Position: CEO & Founder
Company: ZoomInfo
Platform: LinkedIn


Name: Neel Kiran Roshania
Title: Manager
Field: Data Analysis & Applications


Attributes: 
- Name: Jayson Bailey
- Location: Ghana
- Professional Profile: LinkedIn


Attributes:
Name: Jia Huimin 
Company: Shenzhen Chewan Shengbao Wang Engineering Co., Ltd. 
Platform: LinkedIn


In [14]:
#testing curie

title_prompts = ['parse attributes from the string "Henry Schuck - CEO & Founder - ZoomInfo - LinkedIn"',
                 'parse attributes from the string "Yashika Shrikhande - Data Analyst II - ZoomInfo - LinkedIn"',
                 'parse attributes from the string "Neel Kiran Roshania - Manager, Data Analysis & Applications"',
                 'parse attributes from the string "Jayson Bailey - Ghana | Professional Profile | LinkedIn."',
                 'parse attributes from the string "贾会民- 深圳赤湾胜宝旺工程有限公司| LinkedIn - 领英"']

location_prompts = ['parse attributes from the string "63 Mount Rd, Chennai, Tamil Nadu',
                      'parse attributes from the string "McPherson, Kansas, United States"',
                      'parse attributes from the string "Türkiye"',
                      'parse attributes from the string "East Falmouth, Massachusetts, United States"',
                      'parse attributes from the string "Salem, Oregon, United States"']

def complete_prompt(prompt):
    completion = openai.Completion.create(
        engine="text-curie-001",
        prompt=prompt,
        max_tokens=50,
        top_p=0.1,
        n=1,
        stop=None,
        temperature=0.1
    )
    return completion


if __name__ == "__main__":
    with Pool() as pool:
        results = pool.map(complete_prompt, title_prompts)
        for res in results:
            print(res.choices[0].text) #for just the parsed results
#             print(res) #for the entire json object



Henry Schuck - CEO & Founder - ZoomInfo - LinkedIn


Name: Yashika Shrikhande

Title: Data Analyst II

Company: ZoomInfo

LinkedIn: https://www.linkedin.com/in/yashika-shrikhande-a8


Name: Neel Kiran Roshania

Position: Manager, Data Analysis & Applications

Organization:

Name:

Neel Kiran Roshania


LinkedIn Profile:

Name: Jayson Bailey

Location: Ghana

Industry: Professional Profile

LinkedIn Profile:

Name: Jayson Bailey

Location: Ghana

Industry: Professional Profile


name: 贾会民

company: 深圳赤湾胜宝旺工程有限公司

linkedin


In [ ]:
#code structure for possibly testing at scale-

def complete_prompt(prompt):
    completion = openai.Completion.create(
        engine="<your testing model>",
        prompt=prompt,
        max_tokens=50,
        temperature=0.2,
        n=1,
        stop=None
    )
    return completion

results_df= pd.DataFrame()
input_prompts_df = pd.read_csv('<your_sample_input_file>.csv') #a csv or even a snowflake table of input prompts using our snowflake connector
title_input_prompts = input_prompts_df["INPUT_TITLE_PROMPT"] 
title_input_prompts_list = title_input_prompts.values.tolist()

if __name__ == "__main__":
    with Pool() as pool:
        results = pool.map(complete_prompt, title_input_prompts_list)
        output_list = []
        for res in results:
             output_list.append(res.choices[0].text)
                
output_df = pd.DataFrame(output_list, columns=['prompt_result'])                
df["prompt_result"].str.split("\n", expand = True) #example of a possible way to split the output dataframe into attributes. 